# Introduction
Voter Analysis exploration

## Imports
Import libraries and write settings here.

In [95]:
from fastai.tabular import *
from fastai import *

# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 60
pd.options.display.max_rows = 60

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')



# Data Import and Cleaning

In [96]:
#find the data
path = './data/'
df = pd.read_csv('./data/ccesplus.csv',encoding = "ISO-8859-1")

In [97]:
#get rid of spaces and caps in column names (if any)
df.columns = [col.replace(' ','_').lower() for col in df.columns]

In [98]:
# df.head()

# df.columns

# df.describe()

#any duplicate rows?
df.duplicated().sum()

0

In [99]:
#lets see how many columns, how many have no and some missing values
print (f"Total columns={len(df.columns)}")
print(f"Total columns with no Nans= {len(df.columns[~df.isnull().any()])}") 
print(f"Total columns with Nans= {len(df.columns[df.isnull().any()])}") #add ~ to get columns with no missing values

Total columns=866
Total columns with no Nans= 268
Total columns with Nans= 598


The columns with no NaNs (missing values) are the easiest to use since we dont have to impute missing values

## Get a subset of the rows of the data to work on if desired

In [100]:
#get first thousand rows
# df=df[:1000]

## See what the summary columns do
looks like the summaries (..sum) are just the average

In [101]:
columns=['cc.CC16_422c','cc.CC16_422d','cc.CC16_422e','cc.CC16_422f', 'cc.raceviewsum']
columns = [col.lower() for col in columns]
tmp_df=df[columns]
tmp_df.head()

,cc.cc16_422c,cc.cc16_422d,cc.cc16_422e,cc.cc16_422f,cc.raceviewsum
0,1.0,2.0,2.0,2.0,1.75
1,1.0,3.0,1.0,2.0,1.75
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,1.0,1.0,1.00


## Get the columns of interest

In [102]:
#contains all the columns that original logits were run on
#use these to start to compare neural network solution
columns = pd.read_excel('./data/results cces.xlsx',encoding = "ISO-8859-1", skiprows=3)
columns = list(columns.iloc[:,0])

#get the dependant variable(s)
# columns_dep_var= ['cc.TrumpGEVote','cc.TrumpPVote', 'cc.vote12.gop']
columns_dep_var= ['cc.trumpgevote']

#combine
columns.extend(columns_dep_var)

#strip rubbish
columns = [col.replace(' ','_').lower() for col in columns]

## Create dataframe for model training

In [103]:
#select out the columns of interest
df_s=df[columns]

In [104]:
len(df_s.iloc[0])

32

In [105]:
# how many unique values there are per column, 
# use that to guide which columns are categorical and which are continuous
#pick largest value that looks continuous, for instance raceviewsum=17 
#(but see the docs many of theses fields are floats which meand fastai ignores cardinality)
for col in df_s.columns:
    print(f"{col}:{df[col].nunique()}")

cc.blackdum:2
econ.mhi:2646
cc.sex:2
cc.maxeduc.4yr:2
econ.hhpov.p:1030
mort.ucd.despair.disc95.pdpy:2660
cc.union:2
cc.age:80
demo.popdense:2668
cc.emp.nojob:2
rustpc:2437
taa.wrks.disc95.pcpy:2120
cc.faminc:16
crashpc:2626
cc.newsint:5
cc.isimmigrant:2
cc.catholic:2
cc.cc16_305_2:2
job.uer:761
cc.child18:2
cc.evanprot:2
cc.whitedum:2
cc.religiosity:15
cc.cc16_351b:2
cc.cc16_307:4
cc.immviewsum:5
cc.cc16_304:5
cc.inddum:2
cc.ideo7:7
cc.raceviewsum:17
cc.repdum:2
cc.trumpgevote:2


In [106]:
#what types are the columns
df_s.dtypes

cc.blackdum                       int64
econ.mhi                        float64
cc.sex                            int64
cc.maxeduc.4yr                    int64
econ.hhpov.p                    float64
mort.ucd.despair.disc95.pdpy    float64
cc.union                        float64
cc.age                            int64
demo.popdense                   float64
cc.emp.nojob                      int64
rustpc                          float64
taa.wrks.disc95.pcpy            float64
cc.faminc                       float64
crashpc                         float64
cc.newsint                      float64
cc.isimmigrant                    int64
cc.catholic                     float64
cc.cc16_305_2                     int64
job.uer                         float64
cc.child18                      float64
cc.evanprot                     float64
cc.whitedum                       int64
cc.religiosity                  float64
cc.cc16_351b                    float64
cc.cc16_307                     float64


In [107]:
#the percentage of NaNs in each column, note the large percentage of missing values in the bottom columns
#I would guess that cc.faminc and cc.raceviewsum are critical
df_s.isna().sum().sort_values(ascending = True)/len(df)

cc.blackdum                     0.000000
cc.immviewsum                   0.000000
cc.whitedum                     0.000000
cc.cc16_305_2                   0.000000
cc.emp.nojob                    0.000000
cc.age                          0.000000
cc.isimmigrant                  0.000000
cc.sex                          0.000000
cc.maxeduc.4yr                  0.000000
cc.evanprot                     0.000666
cc.newsint                      0.000820
cc.catholic                     0.001130
demo.popdense                   0.001300
econ.hhpov.p                    0.001300
econ.mhi                        0.001300
taa.wrks.disc95.pcpy            0.001300
job.uer                         0.001300
cc.cc16_307                     0.001517
mort.ucd.despair.disc95.pdpy    0.001548
cc.child18                      0.001827
cc.cc16_351b                    0.001950
cc.cc16_304                     0.002059
cc.union                        0.002136
crashpc                         0.002229
rustpc          

In [108]:
#BUMMER! the dependant variable has a lot of missing values (36.7%)
df_s['cc.trumpgevote'].unique()

array([nan,  1.,  0.])

## Remove NaN dependent variable rows
whaddaya going to do?  You have no way of knowing how these people voted

In [109]:
print(f"There are {len(df_s)} rows in df_s")

There are 64600 rows in df_s


In [110]:
#dump the NaN rows
df_s = df_s[pd.notnull(df_s['cc.trumpgevote'] )]

In [111]:
#is the dataset balanced?
df_s['cc.trumpgevote'].value_counts()

0.0    22136
1.0    18754
Name: cc.trumpgevote, dtype: int64

## Split out categorical and continuous variables
see if fastai can auto do it

In [112]:
#give fastai a shot at splitting cat and cont variables
res_cont, res_cat = cont_cat_split(df_s,max_card=18)

In [113]:
??cont_cat_split

In [114]:
print(res_cont)
print(res_cat)

['econ.mhi', 'econ.hhpov.p', 'mort.ucd.despair.disc95.pdpy', 'cc.union', 'cc.age', 'demo.popdense', 'rustpc', 'taa.wrks.disc95.pcpy', 'cc.faminc', 'crashpc', 'cc.newsint', 'cc.catholic', 'job.uer', 'cc.child18', 'cc.evanprot', 'cc.religiosity', 'cc.cc16_351b', 'cc.cc16_307', 'cc.immviewsum', 'cc.cc16_304', 'cc.inddum', 'cc.ideo7', 'cc.raceviewsum', 'cc.repdum', 'cc.trumpgevote']
['cc.blackdum', 'cc.sex', 'cc.maxeduc.4yr', 'cc.emp.nojob', 'cc.isimmigrant', 'cc.cc16_305_2', 'cc.whitedum']


In [115]:
#remove the dependant variable
res_cont = [x for x in res_cont if x not in columns_dep_var]
res_cat = [x for x in res_cat if x not in columns_dep_var]

# Create and Train Model

## Create databunch

In [116]:
#convert trumpgevote to long (otherwise fit fails)
df_s['cc.trumpgevote'] = df_s['cc.trumpgevote'].astype('int64');

In [117]:
# split out data
from sklearn.model_selection import train_test_split
# train,val_test = train_test_split(df_s, test_size=0.2)
# val, test = train_test_split(val_test, test_size=0.5)
# print(str(len(train)))
# print(str(len(val)))
# print(str(len(test)))   
train,tst = train_test_split(df_s, test_size=0.1)
print(str(len(train)))
# print(str(len(val)))
print(str(len(tst)))

36801
4089


In [118]:
len(df_s)

40890

In [119]:
procs = [FillMissing, Categorify, Normalize]

In [120]:
test_long = TabularList.from_df(tst.copy(),  cat_names=res_cat, cont_names=res_cont,procs=procs)
test_vec = TabularList.from_df(tst.copy(),  cat_names=res_cat, cont_names=res_cont,procs=procs)

len(test_long)
len(test_vec)

4089

4089

In [121]:
#all this to get a random list of validation indexes
from numpy import random

#generate a list of all indexes
i = list(range(len(train)))

#shuffle it
random.shuffle(i)

#get number of indexes corresponding to val percentage
number = int(len(i)*.1)

#select validation indexes
val_idx=i[:number]
max(val_idx)

36787

In [171]:
# data = (TabularList.from_df(train,  cat_names=res_cat, cont_names=res_cont, procs=procs)
#                            .split_by_idx(val_idx)
#                            .label_from_df(cols=columns_dep_var)
#                            .databunch())

data = (TabularList.from_df(train,  cat_names=res_cat, cont_names=res_cont, procs=procs)
                           .split_by_idx(val_idx)
                           .label_from_df(cols=columns_dep_var)
                           .add_test(test_vec)
                           .databunch())

In [123]:
# data.show_batch(rows=50)

## Create a learner

In [124]:
#create a learner
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

## Fit the data

In [125]:
learn.fit(epochs=5, lr=1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.221693,0.216459,0.920924
2,0.213995,0.198570,0.918207
3,0.218011,0.199062,0.921467
4,0.204206,0.205722,0.916576
5,0.198874,0.202606,0.924457


# Test the model on test data

## Get a list of correct labels

In [157]:
#lets get a correct list of all the test labels
targets = [int(item['cc.trumpgevote']) for item in test_long]
# targets1 = [int(item[31]) for item in test_long]
# print(f"targets length={len(targets)}, targets1 length={len(targets1)}")
# print(f"Matches in targets and targets1={len([1 for i,x in zip(targets,targets1) if i==x])}")

In [158]:
def eval_accuracy(preds,targs):
    totals = len(preds)
    matches = 0
    for x in zip(preds,targs):
        if x[0]==x[1]:
            matches+=1
    print(f"Got {matches} right out of {totals} samples, Accuracy is {matches/totals}")

## The long, very slow way.  One at a time

In [160]:
#do it the hard way one at a time
preds1=[]
# for item in test_long[:100]:
for item in test_long:
    _,pred, _ = learn.predict(item)
    preds1.append(pred.item())
# len(preds1)
# preds1

In [161]:
eval_accuracy(preds1,targets)

Got 3739 right out of 4089 samples, Accuracy is 0.9144044998777208


## The easy, fast batched way
However note that get_preds does not return the labels as its second param as it indicates in documentation, bug in FastaAI?  

In [162]:
#WARNING THE FOLLOWING CALL, get_preds DOES NOT RETURN CORRECT LABELS AS DOCS IMPLY
predictions, _ = learn.get_preds(DatasetType.Test)

# type(predictions)
# predictions.tolist()[:10]
#targets.tolist()[:100]

In [163]:
#find index of largest output
preds=[torch.argmax(x).item() for x in predictions]
# preds

In [164]:
eval_accuracy(preds,targets)

Got 3739 right out of 4089 samples, Accuracy is 0.9144044998777208


## Miscellaneous checks

In [168]:
#all the same size?
len(test_vec)
len(test_long)
len(preds)
len(preds1)
len(targets)

4089

4089

4089

4089

4089

In [169]:
print(f"Targets same values are ={len([1 for x,y in zip(targets,targets1) if x==y])}")
print(f"Predictions same values are ={len([1 for x,y in zip(preds,preds1) if x==y])}")

Targets same values are =4089
Predictions same values are =4089


In [170]:
from fastai.callbacks import *
from fastai.callbacks.hooks import *
print(model_summary(learn))

Layer (type)         Output Shape         Param #    Trainable 
Embedding            [1, 3]               9          True      
______________________________________________________________________
Embedding            [1, 3]               9          True      
______________________________________________________________________
Embedding            [1, 3]               9          True      
______________________________________________________________________
Embedding            [1, 3]               9          True      
______________________________________________________________________
Embedding            [1, 3]               9          True      
______________________________________________________________________
Embedding            [1, 3]               9          True      
______________________________________________________________________
Embedding            [1, 3]               9          True      
______________________________________________________________

# Results
Show graphs and stats here

# Conclusions and Next Steps
This model is 92% accurate with no data tweaks

# Scratch

In [ ]:
data = test[0]
data[31]
type(data)
data.shape
tmp=learn.predict(data)
len(tmp)
out,pred,res=tmp

out
pred
res

# len(out)
# len(pred)
len(res)